# 2023 Day 12: Hot Springs

## Part 1

You finally reach the hot springs! You can see steam rising from secluded areas attached to the primary, ornate building.

As you turn to enter, the researcher stops you. "Wait - I thought you were looking for the hot springs, weren't you?" You indicate that this definitely looks like hot springs to you.

"Oh, sorry, common mistake! This is actually the onsen! The hot springs are next door."

You look in the direction the researcher is pointing and suddenly notice the massive metal helixes towering overhead. "This way!"

It only takes you a few more steps to reach the main gate of the massive fenced-off area containing the springs. You go through the gate and into a small administrative building.

"Hello! What brings you to the hot springs today? Sorry they're not very hot right now; we're having a lava shortage at the moment." You ask about the missing machine parts for Desert Island.

"Oh, all of Gear Island is currently offline! Nothing is being manufactured at the moment, not until we get more lava to heat our forges. And our springs. The springs aren't very springy unless they're hot!"

"Say, could you go up and see why the lava stopped flowing? The springs are too cold for normal operation, but we should be able to find one springy enough to launch you up there!"

There's just one problem - many of the springs have fallen into disrepair, so they're not actually sure which springs would even be safe to use! Worse yet, their condition records of which springs are damaged (your puzzle input) are also damaged! You'll need to help them repair the damaged records.

In the giant field just outside, the springs are arranged into rows. For each row, the condition records show every spring and whether it is operational (.) or damaged (#). This is the part of the condition records that is itself damaged; for some springs, it is simply unknown (?) whether the spring is operational or damaged.

However, the engineer that produced the condition records also duplicated some of this information in a different format! After the list of springs for a given row, the size of each contiguous group of damaged springs is listed in the order those groups appear in the row. This list always accounts for every damaged spring, and each number is the entire size of its contiguous group (that is, groups are always separated by at least one operational spring: #### would always be 4, never 2,2).

So, condition records with no unknown spring conditions might look like this:

```
#.#.### 1,1,3
.#...#....###. 1,1,3
.#.###.#.###### 1,3,1,6
####.#...#... 4,1,1
#....######..#####. 1,6,5
.###.##....# 3,2,1
```

However, the condition records are partially damaged; some of the springs' conditions are actually unknown (?). For example:

```
???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1
```

Equipped with this information, it is your job to figure out how many different arrangements of operational and broken springs fit the given criteria in each row.

In the first line `(???.### 1,1,3)`, there is exactly one way separate groups of one, one, and three broken springs (in that order) can appear in that row: the first three unknown springs must be broken, then operational, then broken (#.#), making the whole row #.#.###.

The second line is more interesting: `.??..??...?##. 1,1,3` could be a total of four different arrangements. The last ? must always be broken (to satisfy the final contiguous group of three broken springs), and each ?? must hide exactly one of the two broken springs. (Neither ?? could be both broken springs or they would form a single contiguous group of two; if that were true, the numbers afterward would have been 2,3 instead.) Since each ?? can either be #. or .#, there are four possible arrangements of springs.

The last line is actually consistent with ten different arrangements! Because the first number is 3, the first and second ? must both be . (if either were #, the first number would have to be 4 or higher). However, the remaining run of unknown spring conditions have many different ways they could hold groups of two and one broken springs:

```
?###???????? 3,2,1
.###.##.#...
.###.##..#..
.###.##...#.
.###.##....#
.###..##.#..
.###..##..#.
.###..##...#
.###...##.#.
.###...##..#
.###....##.#
```

In this example, the number of possible arrangements for each row is:

- `???.### 1,1,3` - 1 arrangement
- `.??..??...?##. 1,1,3` - 4 arrangements
- `?#?#?#?#?#?#?#? 1,3,1,6` - 1 arrangement
- `????.#...#... 4,1,1` - 1 arrangement
- `????.######..#####. 1,6,5` - 4 arrangements
- `?###???????? 3,2,1` - 10 arrangements

Adding all of the possible arrangement counts together produces a total of 21 arrangements.

For each row, count all of the different arrangements of operational and broken springs that meet the given criteria. What is the sum of those counts?

In [1]:
# Loading the hot spring records (puzzle input) from file
with open('aoc-2023-day-12.txt', 'r') as f:
    raw_hot_spring_records = f.read().splitlines()

# Setting up the sample data
sample_data = '''???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1'''.splitlines()

# Overwriting the hot spring records with the sample data for testing
# raw_hot_spring_records = sample_data

In [2]:
# Instantiating a list to hold the parsed hot spring records
hot_spring_records = []

# Iterating over the raw hot spring records
for line in raw_hot_spring_records:
    schematic = line.split(' ')[0]
    broken_spring_groups = [int(i) for i in line.split(' ')[1].split(',')]
    hot_spring_records.append([schematic, broken_spring_groups])

In [3]:
from itertools import product

def generate_spring_combinations(schematic):
    '''
    Generates all possible combinations for the input schematic
    
    Inputs:
        - schematic (str): The schematic with obscured values
    
    Returns:
        - spring_combinations (list): All possible combinations the schematic can take on
    '''

    # Finding positions of question marks in the string
    positions = [i for i, char in enumerate(schematic) if char == '?']

    # Generating all combinations of replacements for the question marks
    replacement_options = ['.', '#']
    replacements = list(product(replacement_options, repeat=len(positions)))

    # Listing to store the resulting combinations
    spring_combinations = []

    # Replacing question marks with each combination of replacements
    for replacement in replacements:
    
        # Converting the schematic to a list
        schematic_list = list(schematic)
    
        # Replacing each of the question marks with this round's combination
        for idx, pos in enumerate(positions):
            schematic_list[pos] = replacement[idx]
        
        # Appending the new results
        spring_combinations.append(''.join(schematic_list))
        
    return spring_combinations

In [4]:
def count_broken_spring_groups(schematic):
    '''
    Counts the number of broken spring groups in a schematic
    
    Inputs:
        - schematic (str): The current schematic of periods and pound signs
        
    Return:
        - broken_spring_groups (list): A list of values representing the contiguous broken springs
    '''
    # Instantiating a list to hold the number of broken spring groups
    broken_spring_groups = []
    
    # Instantiating a value to represent the current length of the broken spring group
    current_length = 0
    
    # Instantiating a boolean value to see if the last value is part of a group
    in_group = False

    # Iterating over each character in the schematic to count the broken springs (aka pound signs)
    for char in schematic:
        if char == '#':
            if in_group:
                current_length += 1
            else:
                in_group = True
                current_length = 1
        else:
            if in_group:
                broken_spring_groups.append(current_length)
                in_group = False
                current_length = 0

    # Check if the last character was part of a group
    if in_group:
        broken_spring_groups.append(current_length)

    return broken_spring_groups

In [5]:
# Instantiating a value to represent the possible arrangements
possible_arrangements = 0

# Iterating through all the schematics
for hot_spring_record in hot_spring_records:
    
    # Getting the schematic and broken spring group numbers from the record
    schematic = hot_spring_record[0]
    broken_spring_groups = hot_spring_record[1]
    
    # Getting all possible combinations based on current schematics
    spring_combinations = generate_spring_combinations(schematic)
    
    # Iterating over all the combinations
    for combo in spring_combinations:
        
        # Getting the number of broken spring groups in this current combination
        current_broken_spring_groups = count_broken_spring_groups(combo)
        
        # Checking if this is a possible arrangement that works
        if current_broken_spring_groups == broken_spring_groups:
            possible_arrangements += 1

print(f'Number of possible arrangements: {possible_arrangements}')

Number of possible arrangements: 6488
